In [1]:
import numpy as np

In [2]:
x = np.random.randn(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [19]:
# im2col
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    '''다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화)'''
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h) // stride +1
    out_w = (W + 2*pad - filter_w) // stride +1
    
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    
    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
            
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [20]:
x1 = np.random.rand(1,3,7,7)
col1= im2col(x1, 5, 5, stride=1, pad=0)
col1.shape

(9, 75)

In [21]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [22]:
class Convolution():
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out